In [1]:
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Port Lincoln,AU,2021-10-06 17:50:56,-34.7333,135.8667,50.07,61,28,9.26
1,1,Riwaka,NZ,2021-10-06 17:50:57,-41.0833,173.0000,59.00,99,100,3.87
2,2,Naze,JP,2021-10-06 17:50:57,28.3667,129.4833,76.46,75,99,11.74
3,3,Saint-Philippe,RE,2021-10-06 17:50:58,-21.3585,55.7679,71.19,84,41,3.53
4,4,Tanout,NE,2021-10-06 17:50:58,14.9709,8.8879,92.86,14,3,9.26


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
#configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [5]:
#1. Assign the locations to an array of latitude and longitude pairs
locations = city_data_df[["Lat", "Lng"]]
#2. assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
#assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4) 
#add the heatmap layer
fig.add_layer(heat_layer)
#call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#ask the customer to add a minimum and maximum tmeperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp= float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
#filter the dataset to find the cities that fir the criteria.
prefered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &  (city_data_df["Max Temp"] >= min_temp)]

In [10]:
prefered_cities_df.count()

City_ID       179
City          179
Country       179
Date          179
Lat           179
Lng           179
Max Temp      179
Humidity      179
Cloudiness    179
Wind Speed    179
dtype: int64

In [11]:
#create DataFrame called hotel_df to store hotel names along with city, countery, max_temp, and coordinates
hotel_df=prefered_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.tail()

,City,Country,Max Temp,Lat,Lng,Hotel Name
532,Tabunok,PH,80.31,10.7786,124.0122,
533,Miles City,US,81.81,46.4083,-105.8406,
534,Abohar,IN,82.36,30.1500,74.1833,
540,Tual,ID,81.61,-5.6667,132.7500,
542,Hamilton,US,76.59,39.1834,-84.5333,


In [25]:
#set parameters to search for a hotel.
params = {"radius": 5000, "type": "lodging", "key": g_key}

In [32]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
hotel_df.head(20)


,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Naze,JP,76.46,28.3667,129.4833,オーベルジュ・アガルイ
5,Lorengau,PG,77.04,-2.0226,147.2712,Lorengau Harbourside Hotel
7,Hirara,JP,82.53,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
9,Tawala,PH,83.12,9.5542,123.7696,Panglao Regents Park Resort
13,Corn Island,NI,83.50,12.1694,-83.0418,Big Fish Guest House
15,Cairns,AU,75.15,-16.9167,145.7667,Mantra Trilogy Cairns
18,Abidjan,CI,82.36,5.3094,-4.0197,ibis Abidjan Plateau
27,Pedernales,DO,89.29,18.0384,-71.7440,Hotel Costa Marina
28,Kimbe,PG,75.15,-5.5502,150.1429,Kimbe Bay Hotel
29,Puerto Ayora,EC,75.16,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [45]:
#Add a heatmap of temperature for the vacation spots and marker for each city
locations = hotel_df[["Lat","Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

#call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
len(hotel_df["Hotel Name"])

179

In [44]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
#Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]